In [1]:
import os, time
from vllm import LLM, SamplingParams

/opt/app-root/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-19 02:37:56,623	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
%run ../utilities/curl_utilities.ipynb
%run ../utilities/postgres_utilities.ipynb

In [3]:
model_name = "GritLM/GritLM-7B"
number_data_rows = 100

In [4]:
connect_db()

In [5]:
batch_size = 100

In [6]:
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=50)
llm = LLM(model=model_name, max_model_len=4096)

INFO 03-19 02:37:57 __init__.py:207] Automatically detected platform cuda.
INFO 03-19 02:38:03 config.py:549] This model supports multiple tasks: {'score', 'reward', 'embed', 'generate', 'classify'}. Defaulting to 'generate'.
INFO 03-19 02:38:03 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='GritLM/GritLM-7B', speculative_config=None, tokenizer='GritLM/GritLM-7B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), see

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  33% Completed | 1/3 [00:00<00:01,  1.51it/s]
Loading safetensors checkpoint shards:  67% Completed | 2/3 [00:01<00:00,  1.29it/s]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [00:02<00:00,  1.21it/s]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [00:02<00:00,  1.25it/s]



INFO 03-19 02:38:08 model_runner.py:1115] Loading model weights took 13.4967 GB
INFO 03-19 02:38:09 worker.py:267] Memory profiling takes 1.31 seconds
INFO 03-19 02:38:09 worker.py:267] the current vLLM instance can use total_gpu_memory (21.98GiB) x gpu_memory_utilization (0.90) = 19.78GiB
INFO 03-19 02:38:09 worker.py:267] model weights take 13.50GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 0.43GiB; the rest of the memory reserved for KV Cache is 5.79GiB.
INFO 03-19 02:38:10 executor_base.py:111] # cuda blocks: 2966, # CPU blocks: 2048
INFO 03-19 02:38:10 executor_base.py:116] Maximum concurrency for 4096 tokens per request: 11.59x
INFO 03-19 02:38:13 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_ut

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:17<00:00,  2.04it/s]

INFO 03-19 02:38:30 model_runner.py:1562] Graph capturing finished in 17 secs, took 0.26 GiB
INFO 03-19 02:38:30 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 21.72 seconds


In [7]:
sql = f"select description from film limit {number_data_rows}"
film_descriptions = execute_sql_results(sql)

In [8]:
test_description = "Starting batch generative test using local vllm"
print(test_description)
print(len(test_description) * "-") 
tic = time.perf_counter()
print("Texts generated ...", end=" ")
messages = list()
for index, film in enumerate(film_descriptions):
    description = film[0]
    prompt = f"Expand this movie description: {description}"
    messages.append(prompt)

start_offset = 0
end_offset = batch_size
question_response_pairs = list()
while start_offset < number_data_rows:
    print(start_offset, end_offset, end=" ")
    responses = llm.generate(messages[start_offset:end_offset], sampling_params)

    for r in responses:
        original_prompt = r.prompt
        generated_text = r.outputs[0].text
        assert len(generated_text) > 0
        question_response_pairs.append(f"{original_prompt}:{generated_text}")

    start_offset = end_offset
    end_offset = end_offset + batch_size
    if end_offset > number_data_rows:
        end_offset = number_data_rows

assert len(question_response_pairs) == number_data_rows

toc = time.perf_counter()
print("")
print(f"Time to generate text for {number_data_rows} rows: {toc - tic:0.4f} seconds")
print(len(test_description) * "-") 

Starting batch generative test using local vllm
-----------------------------------------------
Texts generated ... 0 100 

Processed prompts: 100%|██████████| 100/100 [00:03<00:00, 28.10it/s, est. speed input: 937.92 toks/s, output: 1403.22 toks/s]


Time to generate text for 100 rows: 3.5883 seconds
-----------------------------------------------


In [9]:
close_db()